In [1]:
# 🚀 DESAFIO TELECOM X - PARTE 2: PREVISÃO DE EVASÃO (CHURN)

# Bibliotecas principais
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configurações
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (8, 5)

# 1. Importação dos dados
url = "https://raw.githubusercontent.com/alura-cursos/challenge2-data-science/refs/heads/main/TelecomX_Data.json"
response = requests.get(url)
data_json = response.json()

# Corrigindo estrutura
df = pd.DataFrame(data_json if isinstance(data_json, list) else data_json)

# 2. Limpeza e preparação
df.columns = df.columns.str.strip()
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Corrigir tipos e remover nulos
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df.dropna(subset=['TotalCharges'], inplace=True)
df.drop_duplicates(inplace=True)

# Criar nova variável
df['Contas_Diarias'] = (df['MonthlyCharges'] / 30).round(2)

# Remover colunas irrelevantes
df.drop(['customerID'], axis=1, inplace=True)

# 3. Codificação de variáveis categóricas
df_encoded = pd.get_dummies(df, drop_first=True)

# 4. Verificar balanceamento
churn_rate = df['Churn'].value_counts(normalize=True) * 100
print("Proporção de Churn:\n", churn_rate)

# 5. Correlação com churn
plt.figure(figsize=(12,10))
corr = df_encoded.corr()
sns.heatmap(corr[['Churn_Yes']].sort_values(by='Churn_Yes', ascending=False), annot=True, cmap='coolwarm')
plt.title('Correlação com a variável Churn')
plt.show()

# 6. Visualizações específicas
sns.boxplot(x='Churn', y='tenure', data=df)
plt.title('Tempo de contrato vs Churn')
plt.show()

sns.boxplot(x='Churn', y='TotalCharges', data=df)
plt.title('Total Gasto vs Churn')
plt.show()

# 7. Separar X e y
X = df_encoded.drop('Churn_Yes', axis=1)
y = df_encoded['Churn_Yes']

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 8. Modelos

## Modelo 1: Regressão Logística (com normalização)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train_scaled, y_train)
y_pred_log = log_model.predict(X_test_scaled)

## Modelo 2: Random Forest (sem normalização)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

# 9. Avaliação

def avaliar_modelo(nome, y_true, y_pred):
    print(f"\n--- {nome} ---")
    print("Acurácia:", accuracy_score(y_true, y_pred).round(4))
    print("Precisão:", precision_score(y_true, y_pred).round(4))
    print("Recall:", recall_score(y_true, y_pred).round(4))
    print("F1-Score:", f1_score(y_true, y_pred).round(4))
    print("Matriz de Confusão:")
    print(confusion_matrix(y_true, y_pred))
    print("Relatório:")
    print(classification_report(y_true, y_pred))

avaliar_modelo("Regressão Logística", y_test, y_pred_log)
avaliar_modelo("Random Forest", y_test, y_pred_rf)

# 10. Análise de importância das variáveis

## Regressão Logística: coeficientes
coef_df = pd.DataFrame(log_model.coef_[0], index=X.columns, columns=['Coeficiente']).sort_values(by='Coeficiente', ascending=False)
print("\nVariáveis mais influentes na Regressão Logística:")
print(coef_df.head(10))

## Random Forest: importâncias
importancias = pd.Series(rf_model.feature_importances_, index=X.columns).sort_values(ascending=False)
print("\nVariáveis mais importantes na Random Forest:")
print(importancias.head(10))

# 11. Relatório Final

print("\n--- RELATÓRIO FINAL ---")

print("\nObjetivo:")
print("Prever a evasão de clientes com base em variáveis demográficas e de uso, usando modelos preditivos.")

print("\nPré-processamento:")
print("- Coluna 'customerID' removida.")
print("- Valores nulos em 'TotalCharges' tratados.")
print("- Variáveis categóricas transformadas com one-hot encoding.")
print("- Criada a variável 'Contas_Diarias'.\n")

print("Análise de Churn:")
print(f"- Taxa de churn: {churn_rate['Yes']:.2f}% dos clientes cancelaram.")
print("- Identificamos forte correlação negativa entre 'tenure' e churn.")
print("- Clientes com contratos mensais e cobranças altas tendem a cancelar mais.\n")

print("Modelos Avaliados:")
print("1. Regressão Logística (com normalização)")
print("2. Random Forest (sem normalização)\n")

print("Comparação:")
print("- Random Forest teve desempenho superior em quase todas as métricas.")
print("- A Regressão Logística apresentou bom desempenho geral e ajuda na interpretação dos fatores.\n")

print("Fatores mais relevantes (Random Forest):")
print(importancias.head(5))

print("\nRecomendações Estratégicas:")
print("- Oferecer benefícios para clientes com contrato mensal migrarem para contratos longos.")
print("- Criar ações de retenção para clientes com menor tempo de casa.")
print("- Revisar planos com mensalidade elevada.")
print("- Acompanhar de perto os clientes com perfis de alto risco (identificados pelos modelos).\n")

print("Fim da análise.")


/tmp/ipython-input-1-998919359.py:32: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


KeyError: 'TotalCharges'